## Load Dependencies

In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
from sibyl import *
from datasets import load_dataset

## Create Datasets

In [4]:
dataset = load_dataset("glue", "sst2", split="train[:10000]")
dataset = dataset.rename_column('sentence', 'text')

Reusing dataset glue (C:\Users\Fabrice\.cache\huggingface\datasets\glue\sst2\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


### Top-n / Bot-n Transforms

In [7]:
top_n = [t[0] for t in [
    ('InsertPunctuationMarks', 0.0),
    ('HomoglyphSwap', 0.0),
    ('AddPositiveEmoji', 0.0),
    ('ImportLinkText', 0.0),
    ('AddNegativeLink', 0.0),
    ('ChangeLocation', 0.0),
    ('ExpandContractions', 0.0),
    ('ChangeNumber', 0.0),
    ('RemovePositiveEmoji', 10.108303249097474),
    ('RemoveNegativeEmoji', 18.75321998969603),
    ('AddNegativeEmoji', 24.52004041764904),
    ('AddNeutralEmoji', 32.76327632763276),
    ('ChangeName', 32.76327632763277),
    ('Demojify', 32.76327632763277),
    ('InsertPositivePhrase', 32.76327632763277),
    ('RemoveNeutralEmoji', 32.76327632763277),
    ('RemoveNegation', 38.51851851851851),
    ('AddPositiveLink', 40.28776978417267),
    ('RandomCharSwap', 43.268945022288264),
    ('TextMix', 43.627646823811425)
]]
black_list = [
    'Concept2Sentence', 
    'ConceptMix', 
    'Emojify', 
    'ImportLinkText', 
    'RemovePositiveEmoji', 
    'RemoveNegativeEmoji', 
    'RemoveNeutralEmoji'
]

In [8]:
top_n_classes = [t for t in TRANSFORMATIONS if t.__name__ in top_n and t.__name__ not in black_list] 
bot_n_classes = [t for t in TRANSFORMATIONS if t.__name__ not in top_n and t.__name__ not in black_list] 

### Sibyl Transformers

In [10]:
top_n_transformer = SibylTransformer("sentiment", transforms=top_n_classes)
bot_n_transformer = SibylTransformer("sentiment", transforms=bot_n_classes)

In [11]:
new_text, new_labels = [], []
for batch in batcher(dataset, 10):
    t_, l_ = top_n_transformer((batch['text'], batch['label']))
    new_text.extend(t_)
    new_labels.extend(l_)
    
df = pd.DataFrame([new_text, new_labels]).T
df.columns = ['text', 'label']
df = df.sample(frac=1).reset_index(drop=True)
df.to_csv('sst2_sibyl_train_top_16.csv')

new_text, new_labels = [], []
for batch in batcher(dataset, 10):
    t_, l_ = bot_n_transformer((batch['text'], batch['label']))
    new_text.extend(t_)
    new_labels.extend(l_)
    
df = pd.DataFrame([new_text, new_labels]).T
df.columns = ['text', 'label']
df = df.sample(frac=1).reset_index(drop=True)
df.to_csv('sst2_sibyl_train_bot_16.csv')